In [1]:
import gdal
import meshplex
import numpy as np
import stripy as stripy
from netCDF4 import Dataset

from gLEM._fortran import defineGTIN

# Create global mesh

Define the refinement level to increase resolution.

In [2]:
ref_lvl = 8
dir_lvl = 'lvl_8'

Build the mesh

In [3]:
grid = stripy.spherical_meshes.icosahedral_mesh(include_face_points=False, 
                                                refinement_levels=ref_lvl)

str_fmt = "{:25} {:9}"
print(str_fmt.format('Number of points', grid.npoints))
print(str_fmt.format('Number of cells', grid.simplices.shape[0]))

Number of points             655362
Number of cells             1310720


In [4]:
radius = 6378137.

coords = np.vstack((grid.points[:,0],grid.points[:,1]))
coords = np.vstack((coords,grid.points[:,2])).T
coords = np.multiply(coords,radius)

Define mesh cells and nodes neighbourhood

In [5]:
Gmesh = meshplex.mesh_tri.MeshTri(coords, grid.simplices)

In [6]:
s = Gmesh.idx_hierarchy.shape
a = np.sort(Gmesh.idx_hierarchy.reshape(s[0], -1).T)
Gmesh.edges = {"nodes": np.unique(a, axis=0)}

In [7]:
ngbNbs, ngbID = defineGTIN(len(coords), Gmesh.cells['nodes'], Gmesh.edges['nodes'])

Store mesh information:

* v: vertex coordinates
* c: triangular cell node index
* n: vertex neighbours for each node

In [8]:
np.savez_compressed(dir_lvl+'/mesh', v=coords, c=grid.simplices, 
                    n=ngbID[:,:6].astype(int))

# Longitude latitude mesh coordinates 

Depending of the resolution of the input data different mesh have to be created...

## Scotese paleo-elevation

In [9]:
elevfile = "data/Scotese-paleomap/60Ma.nc"

In [10]:
data = Dataset(elevfile, "r", format="NETCDF4")
img = np.fliplr(data['z'][:,:].T)

# Convert spherical mesh longitudes and latitudes to degrees
glat=np.mod(np.degrees(grid.lats)+90, 180.0)
glon=np.mod(np.degrees(grid.lons)+180.0, 360.0)

ilons = img.shape[0] * glon / 360.0
ilats = img.shape[1] * glat / 180.0
icoords = np.stack((ilons, ilats))

Store mesh information:

* lat: latitude of each node
* lon: longitude of each node

In [11]:
np.savez_compressed(dir_lvl+'/icoord_elev', v=icoords)

## Scotese paleo-precipitation

In [12]:
rainfile = "data/Scotese-precipitation/60Ma.nc"

In [13]:
data = Dataset(rainfile, "r", format="NETCDF4")
img = np.fliplr(data['z'][:,:].T)

# Convert spherical mesh longitudes and latitudes to degrees
dlons = np.mod(np.degrees(grid.lons)+180.0, 360.0)
dlats = np.mod(np.degrees(grid.lats)+90, 180.0)

# Map mesh coordinates on ETOPO1 dataset
ilons = img.shape[0] * dlons / 360.0
ilats = img.shape[1] * dlats / 180.0

icoords = np.stack((ilons, ilats))

Store mesh information:

* lat: latitude of each node
* lon: longitude of each node

In [14]:
np.savez_compressed(dir_lvl+'/icoord_rain', v=icoords)

## ETOPO1 elevation

In [15]:
etopofile = "data/ETOPO1.tif"

In [16]:
gtiff = gdal.Open(etopofile)

width = gtiff.RasterXSize
height = gtiff.RasterYSize
gt = gtiff.GetGeoTransform()
img = gtiff.GetRasterBand(1).ReadAsArray().T
img = np.fliplr(img)


# Convert spherical mesh longitudes and latitudes to degrees
dlons = np.mod(np.degrees(grid.lons)+180.0, 360.0)
dlats = np.mod(np.degrees(grid.lats)+90, 180.0)

# Map mesh coordinates on ETOPO1 dataset
ilons = img.shape[0] * dlons / 360.0
ilats = img.shape[1] * dlats / 180.0

icoords = np.stack((ilons, ilats))

Store mesh information:

* lat: latitude of each node
* lon: longitude of each node

In [17]:
np.savez_compressed(dir_lvl+'/icoord_etopo', v=icoords)